In [1]:
from flask import Flask, request, jsonify
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
import textwrap

app = Flask(__name__)

summarizer = pipeline('summarization')

def summarize_article(url):
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content, 'html.parser')
    results = soup.find_all(['h1', 'p'])
    text = (result.text for result in results)
    article = ' '.join(text)
    article = article.replace('.', '.<eos>').replace('?', '?<eos>').replace('!', '!<eos>')
    sentences = article.split('<eos>')

    max_chunk = 450
    current_chunk = 0
    chunks = []
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            chunks.append(sentence.split(' '))

    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=40, min_length=10, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    final_summary = ' '.join(summaries)
    wrapped_summary = textwrap.fill(final_summary, width=80)
    return wrapped_summary

@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.get_json()
    url = data['url']
    summary = summarize_article(url)
    return jsonify({'summary': summary})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


No model was supplied, defaulted to google-t5/t5-small and revision d769bba (https://huggingface.co/google-t5/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\rober\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.133.253:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jun/2024 14:00:16] "POST /summarize HTTP/1.1" 400 -
127.0.0.1 - - [12/Jun/2024 14:00:39] "POST /summarize HTTP/1.1" 400 -
Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors
127.0.0.1 - - [12/Jun/2024 14:01:01] "POST /summarize HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2024 14:06:34] "POST /summarize HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2024 11:27:35] "POST /summarize HTTP/1.1" 200 -
